#Importación de Librerias 

Son las librerias necesarias para la ejecución del codigo, con el cual se puede crear y optimizar un portafolio de inversión.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

#Funciones de Rendimiento, riesgo y Sharpe Ratio
Calcula el rendimiento, riesgo y Sharpe Ratio de un portafolio definido

In [ ]:
#Rendimiento promedio
def rend_port(precios,pesos):
  return np.dot(precios.mean(),pesos)*30

#Volatilidad o riesgo
def ries_port(precios,pesos):
  return np.dot(pesos,np.dot(precios.cov()*30,pesos))**0.5

#Sharpe Ratio
def sharper(precios,pesos):
  return rend_port(precios,pesos)/ries_port(precios,pesos)

In [ ]:
#Valor del portafolio
def valor_port(precio,pesos):
  return np.dot(precio,pesos)

#Simulación Monte-Carlo

Se aplica la simulación de Monte-Carlo aun conjunto de emisoras asignadas, y conocer el comportamiento de los distintos portafolio de inversión posibles de construir.

In [ ]:
#Simulador de MonteCarlo para portafolios
def MonteCarlo(precios,num_sim):
  w = np.random.random((num_sim, precios.shape[1]))
  w = (w.T / w.sum(axis=1)).T
  #Calculo de rendimiento y riesgo esperado
  rend_ries_port= [(rend_port(precios,pesos),ries_port(precios,pesos)) for pesos in w]
  rend_ries_port = np.array(rend_ries_port)
  #Calculo del Shape Ratio del portafolio
  shape_p=rend_ries_port[:,0]/rend_ries_port[:,1]
  #Visualización de retabilidad y riesgos de portafolio
  plt.figure(figsize=(10, 6))
  fig = plt.scatter(rend_ries_port[:, 1], rend_ries_port[:, 0],c=shape_p, cmap='coolwarm')
  cb = plt.colorbar(fig)
  cb.set_label('Sharpe ratio')
  plt.xlabel('Volatilidad Esperada')
  plt.ylabel('Rentabilida Esperada')
  plt.title('MonteCarlo - %d Emisoras'%precios.shape[1])  

#Optimización de portafolio

Funciones de optimización de portafolio de inversión a partir de maximizar el Sharpe Ratio, maximizar el rendimiento o minimizar el riesgo del portafolio de inversión.

In [ ]:
def optimizacion(precios):
  #Restricción de la suma de los pesos igual a 1
  cons = {'type': 'eq', 'fun': lambda phi: sum(phi) - 1}
  Num_emi=precios.shape[1]
  #Restricción de pesos, valores positivos y menores que 1
  lim=[(0,1)]*Num_emi
  #Calculo de los Pesos óptimos a través del problema de optimización
  p_opt = minimize(lambda pesos: -sharper(precios, pesos),
                    Num_emi* [1 / Num_emi],
                    bounds=lim,
                    constraints=cons)['x']
  return p_opt

In [ ]:
def maximo_rendimiento(precios):
  #Restricción de la suma de los pesos igual a 1
  #cons = {'type': 'eq', 'fun': lambda phi: sum(phi) - 1}
  cons = ({'type': 'eq', 'fun': lambda phi: sum(phi) - 1},{'type': 'eq', 'fun': lambda phi: ries_port(precios,phi) - 0.0})
  Num_emi=precios.shape[1]
  #Restricción de pesos, valores positivos y menores que 1
  lim=[(0,1)]*Num_emi
  #Calculo de los Pesos óptimos a través del problema de optimización
  p_opt = minimize(lambda pesos: -rend_port(precios, pesos),
                    Num_emi* [1 / Num_emi],
                    bounds=lim,
                    constraints=cons)['x']
  return p_opt

In [ ]:
def minimo_riesgo(precios):
  #Restricción de la suma de los pesos igual a 1
  cons = {'type': 'eq', 'fun': lambda phi: sum(phi) - 1}
  Num_emi=precios.shape[1]
  #Restricción de pesos, valores positivos y menores que 1
  lim=[(0,1)]*Num_emi
  #Calculo de los Pesos óptimos a través del problema de optimización
  p_opt = minimize(lambda pesos: ries_port(precios, pesos),
                    Num_emi* [1 / Num_emi],
                    bounds=lim,
                    constraints=cons)['x']
  return p_opt

#Niveles de rendimiento, riesgo y sharpe ratio

Funciones para gráficar el rendimiento, riesgo y sharpe ratio de un portafolio de inversión, a través del incremento de las emisoras que conforman cada portafolio.

El término de filtro de emisoras son el rendimiento promedio de cada emisora.

In [ ]:
#Optimización Sharpe Ratio
def nivel_SR(datos_rend):
  list_rend=[]
  list_ries=[]
  list_ratio=[]
  for i in range(2,datos_rend.shape[1]+1):
    lista=top(datos_rend,i)
    pesos = optimizacion(datos_rend[lista])
    #print(lista,pesos)
    list_rend.append(rend_port(datos_rend[lista],pesos))
    list_ries.append(ries_port(datos_rend[lista],pesos))
    list_ratio.append(sharper(datos_rend[lista],pesos))
  x=range(2,datos_rend.shape[1]+1)
  print(x,list_rend)
  print(list_ries,'\n',list_ratio)
  fig,ax=plt.subplots(3,1,sharex=True,figsize=(7,7))
  #fig.figure(figsize=(10,5))
  ax[0].plot(x,list_rend)
  ax[1].plot(x,list_ries)
  ax[2].plot(x,list_ratio)
  ax[0].set_title("Rendimientos vs #Acciones")
  ax[1].set_title("Volatilidad vs #Acciones")
  ax[2].set_title("Sharpe Ratio vs #Acciones")
  fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.5)
  plt.show()


In [ ]:
#Optimización Maximizar Rendimiento
def nivel_RM(datos_rend):
  list_rend=[]
  list_ries=[]
  list_ratio=[]
  for i in range(2,datos_rend.shape[1]+1):
    lista=top(datos_rend,i)
    pesos = maximo_rendimiento(datos_rend[lista])
    #print(lista,pesos)
    list_rend.append(rend_port(datos_rend[lista],pesos))
    list_ries.append(ries_port(datos_rend[lista],pesos))
    list_ratio.append(sharper(datos_rend[lista],pesos))
  x=range(2,datos_rend.shape[1]+1)
  print(x,list_rend)
  print(list_ries,'\n',list_ratio)
  fig,ax=plt.subplots(3,1,sharex=True,figsize=(7,7))
  ax[0].plot(x,list_rend)
  ax[1].plot(x,list_ries)
  ax[2].plot(x,list_ratio)
  ax[0].set_title("Rendimientos vs #Acciones")
  ax[1].set_title("Volatilidad vs #Acciones")
  ax[2].set_title("Sharpe Ratio vs #Acciones")
  fig.tight_layout(pad=0.4,w_pad=0.5, h_pad=1.5)
  plt.show()

In [ ]:
#Optimización Maximizar Rendimiento
def nivel_MR(datos_rend):
  list_rend=[]
  list_ries=[]
  list_ratio=[]
  for i in range(2,datos_rend.shape[1]+1):
    lista=top(datos_rend,i)
    pesos = minimo_riesgo(datos_rend[lista])
    #print(lista,pesos)
    list_rend.append(rend_port(datos_rend[lista],pesos))
    list_ries.append(ries_port(datos_rend[lista],pesos))
    list_ratio.append(sharper(datos_rend[lista],pesos))
  x=range(2,datos_rend.shape[1]+1)
  print(x,list_rend)
  print(list_ries,'\n',list_ratio)
  fig,ax=plt.subplots(3,1,sharex=True,figsize=(7,7))
  ax[0].plot(x,list_rend)
  ax[1].plot(x,list_ries)
  ax[2].plot(x,list_ratio)
  ax[0].set_title("Rendimientos vs #Acciones")
  ax[1].set_title("Volatilidad vs #Acciones")
  ax[2].set_title("Sharpe Ratio vs #Acciones")
  fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
  plt.show()

#Optimización a través del volumen

Funciones para gráficar el rendimiento, riesgo y sharpe ratio de un portafolio de inversión, a través del incremento de las emisoras que conforman cada portafolio.

El término de filtro de emisoras son el volumen de transacción de cada emisora.

In [ ]:
#Optimización Sharpe Ratio
def nivel_vol_SR(datos_rend,volumen):
  list_rend=[]
  list_ries=[]
  list_ratio=[]
  for i in range(2,datos_rend.shape[1]+1):
    lista=top(volumen,i)
    pesos = optimizacion(datos_rend[lista])
    #print(lista,pesos)
    list_rend.append(rend_port(datos_rend[lista],pesos))
    list_ries.append(ries_port(datos_rend[lista],pesos))
    list_ratio.append(sharper(datos_rend[lista],pesos))
  x=range(2,datos_rend.shape[1]+1)
  print(x,list_rend)
  print(list_ries,'\n',list_ratio)
  fig,ax=plt.subplots(3,1,sharex=True,figsize=(7,7))
  #fig.figure(figsize=(10,5))
  ax[0].plot(x,list_rend)
  ax[1].plot(x,list_ries)
  ax[2].plot(x,list_ratio)
  ax[0].set_title("Rendimientos vs #Acciones")
  ax[1].set_title("Volatilidad vs #Acciones")
  ax[2].set_title("Sharpe Ratio vs #Acciones")
  fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.5)
  plt.show()


In [ ]:
#Optimización Maximizar Rendimiento
def nivel_vol_RM(datos_rend,volumen):
  list_rend=[]
  list_ries=[]
  list_ratio=[]
  for i in range(2,datos_rend.shape[1]+1):
    lista=top(volumen,i)
    pesos = maximo_rendimiento(datos_rend[lista])
    #print(lista,pesos)
    list_rend.append(rend_port(datos_rend[lista],pesos))
    list_ries.append(ries_port(datos_rend[lista],pesos))
    list_ratio.append(sharper(datos_rend[lista],pesos))
  x=range(2,datos_rend.shape[1]+1)
  print(x,list_rend)
  print(list_ries,'\n',list_ratio)
  fig,ax=plt.subplots(3,1,sharex=True,figsize=(7,7))
  ax[0].plot(x,list_rend)
  ax[1].plot(x,list_ries)
  ax[2].plot(x,list_ratio)
  ax[0].set_title("Rendimientos vs #Acciones")
  ax[1].set_title("Volatilidad vs #Acciones")
  ax[2].set_title("Sharpe Ratio vs #Acciones")
  fig.tight_layout(pad=0.4,w_pad=0.5, h_pad=1.5)
  plt.show()

In [ ]:
#Optimización Maximizar Rendimiento
def nivel_vol_MR(datos_rend,volumen):
  list_rend=[]
  list_ries=[]
  list_ratio=[]
  for i in range(2,datos_rend.shape[1]+1):
    lista=top(volumen,i)
    pesos = minimo_riesgo(datos_rend[lista])
    #print(lista,pesos)
    list_rend.append(rend_port(datos_rend[lista],pesos))
    list_ries.append(ries_port(datos_rend[lista],pesos))
    list_ratio.append(sharper(datos_rend[lista],pesos))
  x=range(2,datos_rend.shape[1]+1)
  print(x,list_rend)
  print(list_ries,'\n',list_ratio)
  fig,ax=plt.subplots(3,1,sharex=True,figsize=(7,7))
  ax[0].plot(x,list_rend)
  ax[1].plot(x,list_ries)
  ax[2].plot(x,list_ratio)
  ax[0].set_title("Rendimientos vs #Acciones")
  ax[1].set_title("Volatilidad vs #Acciones")
  ax[2].set_title("Sharpe Ratio vs #Acciones")
  fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
  plt.show()